Železniške povezave so podane v obliki slovarja. Ključi so imena postaj (npr. Zidani most), pripadajoče vrednosti pa seznami povezav na neposredno povezane postaje. Povezava je trojka z imenom ciljne postaje (npr. Laško), časom odhoda (v minutah od polnoči) s postaje v ključu (Zidani most) in časom prihoda na ciljno postajo (npr. Laško). Če ključu `"Zidani Most"` pripada seznam `[('Laško', 364, 382), ('Hrastnik', 365, 386), ('Laško', 375, 390), ('Radeče', 379, 395)]`, iz Zidanega Mostu odhaja vlak v Laško ob 6:04 in prispe tja ob 6:22, nato v Hrastnik ob 6:05 s prihodom ob 6:26, potem spet v Laško ob 6:15 s prihodom ob 6:30, potem v Radeče ob 6:19 s prihodom ob 6:35. **Povezave so urejene po časih odhodov.** Časi so izmišljeni in vsaka povezava z resničnimi prihodi in odhodi vlakov je zgolj naključna. V tem smislu so podobni dejanskim voznim redom Slovenskih železnic.

## 1. Postaje

Napišite funkcijo `postaje(povezave)`, ki prejme slovar s povezavami in vrne tri množice: množico končnih postaj, množico prehodnih postaj in množico križišč. Postaja je križišče, če z nje vozijo vlaki na tri ali več različnih postaj. Postaja je prehodna, če je povezana z dvema različnima postajama, in končna, če vlaki z nje vozijo na eno samo postajo. Vse povezave so dvosmerne: če vozi vlak iz A v B, vozi tudi iz B v A.

### Rešitev

Začeli smo torej z nalogo iz slovarjev in množic.

Funkcija bo v vsakem primeru sestavljena iz dveh delov. V prvem bomo šli čez vse povezave in pripravili slovar, katerega ključi bodo povezave, vrednosti pa množice postaj, s katerimi je ta postaja povezana. V drugem delu bomo šli čez ta slovar in njegove ključe (imena postaj) metali v tri množice), glede na to, koliko elementov je v množici, ki pripada temu ključu.

In [1]:
def postaje(povezave):
    navezave = {}
    for postaja, odhodi in povezave.items():
        navezave[postaja] = set()
        for ciljna, _, _ in odhodi:
            navezave[postaja].add(ciljna)
    koncne = set()
    prehodne = set()
    krizisca = set()
    for postaja, povezane in navezave.items():
        if len(povezane) == 1:
            koncne.add(postaja)
        elif len(povezane) == 2:
            prehodne.add(postaja)
        else:
            krizisca.add(postaja)
    return koncne, prehodne, krizisca

Tudi `defaultdict` bi nam prihranil eno vrstico, `navezave[postaja] = set()`, hkrati pa dodal en `import`, zato tu nima posebnega smisla. Tu namreč točno vemo, kdaj moramo dodati postajo v slovar navezave: vedno, takoj znotraj zunanje zanke.

Kdor zna malo več, lahko naredi

In [2]:
def postaje(povezave):
    navezave = {}
    for postaja, odhodi in povezave.items():
        navezave[postaja] = {ciljna for ciljna, _, _ in odhodi}
    koncne = {k for k, v in navezave.items() if len(v) == 1}
    prehodne = {k for k, v in navezave.items() if len(v) == 2}
    krizisca = set(navezave) - prehodne - koncne
    return koncne, prehodne, krizisca

Kdor zna to uporabiti še malo boljše, pa napiše celo

In [3]:
def postaje(povezave):
    navezave = {postaja: {ciljna for ciljna, _, _ in odhodi}
                for postaja, odhodi in povezave.items()}
    koncne = {k for k, v in navezave.items() if len(v) == 1}
    prehodne = {k for k, v in navezave.items() if len(v) == 2}
    krizisca = set(navezave) - prehodne - koncne
    return koncne, prehodne, krizisca

## 2. Potovalni časi

Napišite funkcijo `naslednja_povezava(povezave, odkod, kam, cas)`, ki vrne par s časom, ko odide naslednji vlak ob podanem času cas (ali po njem) iz odkod v kam, in časom prihoda v kam. Klic `naslednja_povezava(povezave, "Zidani Most", "Laško", 370)` vrne `(375, 390)`. Če vlaka po podanem času ni, vrne `None`.

Napišite funkcijo `potovalni_cas(povezave, pot, zacetek)`, ki vrne čas (v minutah), ki ga bo potnik potreboval, da prepotuje podano pot (seznam imen postaj), če se nanjo odpravi ob času zacetek. Potnik na vsaki postaji uporabi prvi možni vlak do naslednje postaje. Če poti ni možno opraviti, ker določena povezava ne obstaja ali pa na tisti dan ni več vlaka na tej povezavi, funkcije vrne None. Brez skrbi smete predpostaviti, da vlaki Slovenskih železnic nimajo zamud.

Klic `potovalni_cas(povezave, ["Kranj", "Škofja Loka", "Ljubljana", "Litija", "Hrastnik"], 420)` vrne `165`. Potnik ob 439 sede na vlak Kranj – Škofja Loka, kamor prispe ob 464 in takoj nadaljuje v Ljubljano s prihodom ob 480. Do 542 čaka na vlak, ki ga ob 565 pripelje v Litijo, takoj nadaljuje v Hrastnik in prispe ob 585. 585 – 420 = 165.


### Rešitev

Prva funkcija je preprosta: potrebno je iti prek odhodov s posamične postaje (za kar moramo le znati priti do vrednosti, ki pripada določenemu ključu v slovarju -- se pravi, znati moramo uporabiti podani slovar) in poiskati povezavo, ki vodi tja, kamor želimo, takrat, ko želimo (ali kasneje).

In [6]:
def naslednja_povezava(povezave, odkod, kam, cas):
    for k, odhod, prihod in povezave[odkod]:
        if k == kam and odhod >= cas:
            return odhod, prihod
    return None

Druga funkcija zahteva, da znamo narediti zanko čez seznam parov v slovarju -- nekaj, kar smo naredili na praktično vsakem predavanju. Za vsak par pokličemo prejšnjo funkcijo in sproti beležimo čas, ko se znajdemo na določeni povezavi. Če prejšnja funkcija vrne `None`, pa je veselja konec.

In [4]:
from itertools import pairwise

def potovalni_cas(povezave, pot, zacetek):
    cas = zacetek
    for odkod, kam in pairwise(pot):
        casa = naslednja_povezava(povezave, odkod, kam, cas)
        if casa is None:
            return None
        cas = casa[1]
    return cas - zacetek

## 3. Vozni redi

Vozni redi so v resnici sestavljeni tako, da posamični vlak prevozi določeno relacijo, na primer Ljubljana – Škofja Loka – Kranj – Jesenice. Vsak vlak, ki gre iz Ljubljane v Škofjo Loko, že ob isti uri nadaljuje pot v Kranj in nato do Jesenic: če nek vlak prispe ob 800 iz Ljubljana v Škofjo Loko, odide nek vlak ob 800 iz Škofje Loke v Kranj.

Napišite funkcijo `vozni_red(povezave, linija, ime_datoteke)`, ki prejme slovar povezav, seznam linija, ki vsebuje imena krajev (npr `["Ljubljana", "Škofja Loka", "Kranj", "Jesenice"]`) in ime datoteke. V datoteko mora izpisati vozni red, kot je izpisan spodaj. Imena postaj so izpisana na 20 mest in pred vsakim časom sta dva presledka.

```
           Ljubljana  05:59  06:58  07:57  08:53  09:53  10:57  11:54  12:54  13:55  14:59  15:54
         Škofja Loka  06:18  07:15  08:21  09:08  10:16  11:20  12:09  13:14  14:11  15:15  16:16
               Kranj  06:36  07:31  08:42  09:24  10:34  11:41  12:26  13:33  14:27  15:35  16:39
            Jesenice  06:54  07:56  08:57  09:42  10:59  11:59  12:47  13:50  14:48  15:55  16:55
```

(Za razmislek: druga vrstica, recimo, pravzaprav vsebuje ravno ure vseh odhodov iz Škofje Loke v Kranj, ki so hkrati tudi ure vseh prihodov iz Ljubljane v Škofjo Loko. Prva vrstica pa seveda vsebuje le (vse) odhode in zadnja vse prihode.)

### Rešitev

Tole je bila za moj okus najbolj zoprna naloga. Sploh ne težka, temveč preprosto nadležna. :)

Rešitev je lahko takšna:

In [12]:
def vozni_red(povezave, linija, ime_dat):
    f = open(ime_dat, "wt", encoding="utf-8")
    
    for odkod, kam in pairwise(linija):
        casi = []
        for k, odhod, prihod in povezave[odkod]:
            if k == kam:
                casi.append(f"{odhod // 60:0>2}:{odhod % 60:0>2}")
        f.write(f'{odkod:>20}  {"  ".join(casi)}\n')

    casi = []
    for k, odhod, prihod in povezave[odkod]:
        if k == kam:
            casi.append(f"{prihod // 60:0>2}:{prihod % 60:0>2}")
    f.write(f'{kam:>20}  {"  ".join(casi)}\n')

V prvi zanki gremo spet čez vse pare postaj na liniji; imenovali ju bomo `odkod` in `kam`. V seznam `casi` zbiramo čase, ki jih bomo kasneje izpisali kot odhodne čase s postaje `odkod` v `kam`. Čase že tako oblikujemo v primerne nize: uro dobimo s celoštevilskim deljenjem s 60, minute so ostanek po deljenju, oboje izpišemo na dve mesti z vodilno ničlo.

Ko v notranji zanki naberemo vse čase, v datoteko zapišemo ime postaje (na 20 mest) nato dva presledka in nato s presledki ločene čase. Gre tudi drugače; tu sem pač naredil tako.

Zoprno je, da nam po tem manjka zadnja vrstica, zato moramo ponoviti kodo iz zanke, le da jo spremenimo tako, da namesto odhodov s predzadnje postaje na zadnjo zbiramo prihode na zadnjo postajo s predzadnje.

Nič težkega, samo malo nadležno.

## 4. Izračun poti

Napišite funkcijo `cas_prihoda(povezave, odkod, kam, zacetek, omejitev)`, ki prejme slovar, začetno in končno postajo, čas, ko se bo nekdo odpravil na pot s postaje odkod in skrajni čas, do katerega mora prispeti. Funkcija mora vrniti najzgodnejši čas, ko lahko pride do kam, ob predpostavki, da poišče optimalno zaporedje postaj, da mu prestopanja ne vzamejo časa in da vlaki ne zamujajo. :) Primer najdete v testih.

Pomemben spoiler: testi sicer nastavijo omejitev na neko veliko število. Ko vaša funkcija že najde neko možno pot, pa naj jo v rekurzivnem klicu zaostri, da znotraj njih ne bo iskala poti, ki bi vzele več časa kot najkrajša pot, ki ste jo našli doslej. Brez tega bo funkcija prepočasna.

### Rešitev

In [5]:
def cas_prihoda(povezave, odkod, kam, cas, omejitev):
    if odkod == kam:
        return cas
    najkrajsi = None
    for vmesna, odhod, prihod in povezave[odkod]:
        if odhod >= cas and prihod <= omejitev:
            cp = cas_prihoda(povezave, vmesna, kam, prihod, omejitev)
            if cp is not None and (najkrajsi is None or cp < najkrajsi):
                najkrajsi = cp
                omejitev = najkrajsi
    return najkrajsi

Če smo že tam, kamor moramo priti, vrnemo trenutni čas.

Sicer gremo prek odhodov s trenutne postaje. Za vse odhode - na poljubno postajo, ki so kasnejši od trenutnega časa, vendar na naslednjo postajo pridejo pred končno omejitvijo, preverimo, kdaj bi s te, poljubne vmesne postaje, lahko prišli na končno - upoštevši, trenutni čas in trenutno omejitev. To seveda naredimo z rekurzivnim klicem.

Če se izkaže, da je ta čas zgodnejši od najzgodnejšega doslej, popravimo najzgodnejši čas **in omejitev!** To navodilo iz naloge je pomembno, saj s tem bistveno omejimo čas iskanja. Brez tega ... pravzaprav ne vem, koliko časa bi trajali testi, saj se mi ni dalo čakati.

## 5. Potnik

Napišite razred `Potnik`. Razred naj predpostavi, da se povezave nahajajo v globalni spremenljivki z imenom `povezave`.

- Konstruktor prejme začetno postajo in trenutni čas.
- Metoda `premik(kam)` prejme postajo, na katero potnik potuje s postaje, na kateri se trenutno nahaja. Premik opravi v najzgodnejšem možnem času. Če določenega premika ni možno opraviti (ker povezava ne obstaja ali pa je potnik zamudil zadnjo povezavo v podanem dnevu), funkcija ne naredi ničesar.
- Metoda `kje()` vrne postajo, na kateri se trenutno nahaja potnik, in čas, ko je prispel nanjo.
- Metoda `izguba()` vrne skupni čas čakanja na povezave (ne skupni čas potovanja!).

### Rešitev

Naloga iz rekurzije je bila tokrat malenkost težja kot običajno (ja, zaradi argumenta `omejitve`), za kompenzacijo pa vam je bila naloga iz objektnega programiranja praktično podarjena. Večini je ta kompromis najbrž ustrezal. :)

In [6]:
class Potnik:
    def __init__(self, postaja, cas):
        self.postaja = postaja
        self.cas = cas
        self.cakanje  = 0

    def premik(self, kam):
        cas = naslednja_povezava(povezave, self.postaja, kam, self.cas)
        if cas is not None:
            self.cakanje += cas[0] - self.cas
            self.postaja = kam
            self.cas = cas[1]

    def kje(self):
        return self.postaja, self.cas

    def izguba(self):
        return self.cakanje

Kot vedno se je potrebno le odločiti, kaj bomo shranjevali v atribute razreda in tokrat je bilo to očitno: natančno tisto, kar vračata metodi `kje` in `izguba`. Metoda premik pa le poišče najzgodnejši odhod (pri čemer si lahko pomagamo kar s funkcijo, ki jo že imamo) in ustrezno spremeni vrednosti atributov.